In [1]:
from google.colab import drive

In [2]:
drive.mount('/content/drive/')

Mounted at /content/drive/


In [3]:
cd drive/MyDrive/Kaggle/Google\ Brain\ Ventilator\ Pressure/ventilator-pressure

/content/drive/MyDrive/Kaggle/Google Brain Ventilator Pressure/ventilator-pressure


In [4]:
import notebook_module
notebook_module.execute_hook()

/usr/local/lib/python3.7/dist-packages/nbformat/current.py:19: UserWarning: nbformat.current is deprecated.

- use nbformat for read/write/validate public API
- use nbformat.vX directly to composing notebooks of a particular version

  """)


In [17]:
import pandas as pd
import data_preparation

from xgboost import XGBRegressor

from sklearn.model_selection import train_test_split

import numpy as np

import pickle

import os

In [6]:
df = pd.read_csv('data/train.csv', index_col=0)

/usr/local/lib/python3.7/dist-packages/numpy/lib/arraysetops.py:580: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [7]:
df = data_preparation.xgboost_data_preparation(df)

In [8]:
df.head()

,breath_id,time_step,u_in,u_out,pressure,R_5,R_20,R_50,C_10,C_20,C_50
id,,,,,,,,,,,
1,1,0.000000,0.083334,0,5.837492,0,1,0,0,0,1
2,1,0.033652,18.383041,0,5.907794,0,1,0,0,0,1
3,1,0.067514,22.509278,0,7.876254,0,1,0,0,0,1
4,1,0.101542,22.808822,0,11.742872,0,1,0,0,0,1
5,1,0.135756,25.355850,0,12.234987,0,1,0,0,0,1


## Reshape Data in to Row Vectors

In [9]:
df_reshape_values = df[['time_step', 'u_in', 'u_out']]
reshaped_arr = df_reshape_values.values.reshape(len(df_reshape_values)//80, 80*3)

In [10]:
lung_data_arr = df.loc[df['time_step'] == 0, ['R_5', 'R_20', 'R_50', 'C_10', 'C_20', 'C_50']].values

In [11]:
columns = [[f'time_step_{x}', f'u_in_{x}', f'u_out_{x}'] for x in range(1,81)]
columns = np.array(columns).reshape(80*3)
columns = np.append(columns, ['R_5', 'R_20', 'R_50', 'C_10', 'C_20', 'C_50'])

data = np.column_stack((reshaped_arr, lung_data_arr))

df_reshaped = pd.DataFrame(data, columns=columns, index=df['breath_id'].unique())
df_reshaped.index.name = 'breath_id'

In [12]:
df_reshaped

,time_step_1,u_in_1,u_out_1,time_step_2,u_in_2,u_out_2,time_step_3,u_in_3,u_out_3,time_step_4,u_in_4,u_out_4,time_step_5,u_in_5,u_out_5,time_step_6,u_in_6,u_out_6,time_step_7,u_in_7,u_out_7,time_step_8,u_in_8,u_out_8,time_step_9,u_in_9,u_out_9,time_step_10,u_in_10,u_out_10,time_step_11,u_in_11,u_out_11,time_step_12,u_in_12,u_out_12,time_step_13,u_in_13,u_out_13,time_step_14,...,u_out_69,time_step_70,u_in_70,u_out_70,time_step_71,u_in_71,u_out_71,time_step_72,u_in_72,u_out_72,time_step_73,u_in_73,u_out_73,time_step_74,u_in_74,u_out_74,time_step_75,u_in_75,u_out_75,time_step_76,u_in_76,u_out_76,time_step_77,u_in_77,u_out_77,time_step_78,u_in_78,u_out_78,time_step_79,u_in_79,u_out_79,time_step_80,u_in_80,u_out_80,R_5,R_20,R_50,C_10,C_20,C_50
breath_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.083334,0.0,0.033652,18.383041,0.0,0.067514,22.509278,0.0,0.101542,22.808822,0.0,0.135756,25.355850,0.0,0.169698,27.259866,0.0,0.203708,27.127486,0.0,0.237723,26.807732,0.0,0.271776,27.864715,0.0,0.305732,28.313036,0.0,0.339675,26.866758,0.0,0.373680,26.762803,0.0,0.407650,27.993274,0.0,0.441804,...,1.0,2.349457,4.929162,1.0,2.383458,4.940237,1.0,2.417553,4.949604,1.0,2.451473,4.957466,1.0,2.485587,4.964136,1.0,2.519502,4.969730,1.0,2.553593,4.974474,1.0,2.587754,4.978481,1.0,2.621773,4.981847,1.0,2.655746,4.984683,1.0,2.689766,4.987079,1.0,0.0,1.0,0.0,0.0,0.0,1.0
2,0.0,12.184338,0.0,0.033998,13.980205,0.0,0.068151,12.574163,0.0,0.102062,11.730537,0.0,0.135917,12.124229,0.0,0.170010,11.055636,0.0,0.204087,10.774428,0.0,0.238071,11.111878,0.0,0.272077,10.099527,0.0,0.306069,10.043285,0.0,0.339961,9.424626,0.0,0.374056,9.424626,0.0,0.407880,9.030934,0.0,0.442127,...,1.0,2.349803,4.936892,1.0,2.384037,4.946820,1.0,2.418010,4.955128,1.0,2.452122,4.962164,1.0,2.486156,4.968085,1.0,2.520255,4.973087,1.0,2.554363,4.977307,1.0,2.588413,4.980860,1.0,2.622476,4.983857,1.0,2.656502,4.986382,1.0,2.690624,4.988518,1.0,0.0,1.0,0.0,0.0,1.0,0.0
3,0.0,0.000000,0.0,0.031964,7.187242,0.0,0.063952,13.338781,0.0,0.095892,17.111335,0.0,0.127871,18.353980,0.0,0.159838,17.417672,0.0,0.191693,15.353111,0.0,0.223502,13.641899,0.0,0.255286,12.336795,0.0,0.287100,11.487085,0.0,0.319042,11.166449,0.0,0.350995,10.814289,0.0,0.382995,9.949995,0.0,0.414961,...,1.0,2.206242,4.854806,1.0,2.238075,4.876171,1.0,2.270010,4.894445,1.0,2.301942,4.910021,1.0,2.333896,4.923308,1.0,2.365813,4.934620,1.0,2.397749,4.944269,1.0,2.429649,4.952485,1.0,2.461564,4.959494,1.0,2.493380,4.965451,1.0,2.525398,4.970562,1.0,0.0,0.0,1.0,0.0,1.0,0.0
4,0.0,0.000000,0.0,0.031854,1.262539,0.0,0.063761,4.001352,0.0,0.096498,6.379411,0.0,0.128229,8.295194,0.0,0.160056,9.790410,0.0,0.191822,10.874443,0.0,0.224365,13.336506,0.0,0.256180,15.998234,0.0,0.287979,18.294006,0.0,0.319634,20.196161,0.0,0.352385,21.801582,0.0,0.384078,23.023690,0.0,0.415823,...,1.0,2.208014,4.858861,1.0,2.239768,4.879581,1.0,2.272595,4.897809,1.0,2.304395,4.912832,1.0,2.336085,4.925605,1.0,2.367802,4.936515,1.0,2.400609,4.946119,1.0,2.432266,4.954007,1.0,2.464062,4.960767,1.0,2.495845,4.966532,1.0,2.528579,4.971585,1.0,0.0,0.0,1.0,0.0,0.0,1.0
5,0.0,21.424375,0.0,0.033967,28.504653,0.0,0.068797,29.353025,0.0,0.102813,30.502095,0.0,0.136925,31.024647,0.0,0.172141,32.224651,0.0,0.206255,32.364788,0.0,0.240303,32.858346,0.0,0.275260,32.551761,0.0,0.309354,34.031274,0.0,0.343662,32.964679,0.0,0.378789,33.533876,0.0,0.413108,32.437493,0.0,0.447366,...,1.0,2.380527,4.945768,1.0,2.414673,4.954280,1.0,2.449685,4.961622,1.0,2.483980,4.967670,1.0,2.518057,4.972735,1.0,2.553102,4.977117,1.0,2.587415,4.980725,1.0,2.621570,4.983751,1.0,2.656784,4.986374,1.0,2.690775,4.988504,1.0,2.724876,4.990306,1.0,1.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,..

# Multiregression Model

In [27]:
df_train = df_reshaped.copy()    

for i in range(80):
  target = df.groupby('breath_id')['pressure'].nth(i)
  
  df_train['pressure'] = target.values

  X, X_val, y, y_val = train_test_split(df_train.drop('pressure', axis=1), df_train['pressure'], test_size=0.2)
  model = XGBRegressor(tree_method='gpu_hist', objective='reg:squarederror', max_depth=10, n_estimators=100000, learning_rate=0.02)
  model.fit(X, y, eval_set=[(X_val, y_val)], eval_metric='mae', verbose=0, early_stopping_rounds=200)

  predictions = model.predict(df_train.drop('pressure', axis=1))

  df_train[f'pressure_step_{i}'] = predictions

  filename = os.path.join(os.path.abspath(''), 'Multiregression Models', f'model_{i}.pickle.dat')
  with open(filename, 'wb') as f:
    pickle.dump(model, f)

In [43]:
i = 24
filename = os.path.join(os.path.abspath(''), 'Multiregression Models', f'model_{i}.pickle.dat')
with open(filename, 'rb') as f:
  model = pickle.load(f)

In [44]:
model.best_score

0.250376